In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# Sample data
# Each sequence has three time steps, and we want to predict the next item in the sequence
# The sequences are represented as tensors
data = torch.tensor([[[1.0], [2.0], [3.0]],
                     [[4.0], [5.0], [6.0]],
                     [[7.0], [8.0], [9.0]]])

data =  pd.read_csv('../datasets/sequential/carsii_timedelta_rand_seq/context/test.csv', index=False)

# LSTM Model
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        output = self.fc(lstm_out[:, -1, :])  # Take the last time step output
        return output

# Model, Loss, and Optimizer
input_size = 1  # The size of each element in the sequence
hidden_size = 2  # Number of hidden units in the LSTM
output_size = 1  # Output size, predicting one value
model = SimpleLSTM(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training
epochs = 1000
for epoch in range(epochs):
    # Forward pass
    outputs = model(data)

    # Ground truth labels (in this case, predicting the next item in the sequence)
    targets = torch.tensor([[4.0], [7.0], [10.0]])

    # Compute the loss
    loss = criterion(outputs, targets)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Testing
with torch.no_grad():
    test_data = torch.tensor([[[10.0], [11.0], [12.0]]])
    predicted_output = model(test_data)
    print(f'Predicted next item in the sequence: {predicted_output.item()}')


Epoch [100/1000], Loss: 11.5962
Epoch [200/1000], Loss: 4.8652
Epoch [300/1000], Loss: 1.9562
Epoch [400/1000], Loss: 1.0129
Epoch [500/1000], Loss: 0.3090
Epoch [600/1000], Loss: 0.0685
Epoch [700/1000], Loss: 0.0132
Epoch [800/1000], Loss: 0.0021
Epoch [900/1000], Loss: 0.0003
Epoch [1000/1000], Loss: 0.0000
Predicted next item in the sequence: 10.03114128112793


In [9]:
import pandas as pd
data =  pd.read_csv('./datasets/sequential/carsii_timedelta_rand_seq/context/test.csv', parse_dates=['datetime'])
data = data[data['session_id'] == 0]
data = data[data['KBI_speed'] != 0]['KBI_speed']

data

63      0.434495
64      1.600000
66      0.185019
67      5.718154
68      7.736621
          ...   
2845    3.352508
2846    1.280000
2869    0.300077
2870    1.920000
2871    0.439339
Name: KBI_speed, Length: 2469, dtype: float64

In [12]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn

class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        return h_n[-1]

class TransformerModel(nn.Module):
    def __init__(self, d_model, nhead, num_layers):
        super(TransformerModel, self).__init__()
        self.transformer = nn.Transformer(d_model, nhead, num_layers)

    def forward(self, src, tgt):
        return self.transformer(src, tgt)


class ConcatenatedModel(nn.Module):
    def __init__(self, lstm_input_size, lstm_hidden_size, transformer_d_model, transformer_nhead, transformer_layers, fc_hidden_size, output_size):
        super(ConcatenatedModel, self).__init__()

        # LSTM Encoder
        self.lstm_encoder = EncoderLSTM(lstm_input_size, lstm_hidden_size)

        # Transformer Model
        self.transformer_model = TransformerModel(transformer_d_model, transformer_nhead, transformer_layers)

        # Fully Connected Layers
        self.fc_layers = nn.Sequential(
            nn.Linear(lstm_hidden_size + transformer_d_model, fc_hidden_size),
            nn.ReLU(),
            nn.Linear(fc_hidden_size, fc_hidden_size),
            nn.ReLU(),
            nn.Linear(fc_hidden_size, output_size)
        )

    def forward(self, lstm_input, transformer_input):
        # LSTM Encoding
        lstm_encoded = self.lstm_encoder(lstm_input)

        # Transformer Encoding
        transformer_encoded = self.transformer_model(transformer_input)

        # Concatenate the representations
        concatenated_rep = torch.cat((lstm_encoded, transformer_encoded), dim=1)

        # Fully Connected Layers
        output = self.fc_layers(concatenated_rep)
        return output


# Generate synthetic data
def generate_data(num_samples, sequence_length, feature_dim):
    lstm_input = torch.rand((num_samples, sequence_length, feature_dim))
    transformer_input = torch.randint(0, 2, (num_samples, sequence_length))  # Binary movie clicks
    labels = torch.randint(0, 2, (num_samples,))  # Binary labels (for example)

    return lstm_input, transformer_input, labels

# Custom dataset
class CustomDataset(Dataset):
    def __init__(self, num_samples, sequence_length, feature_dim):
        self.lstm_input, self.transformer_input, self.labels = generate_data(num_samples, sequence_length, feature_dim)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'lstm_input': self.lstm_input[idx], 'transformer_input': self.transformer_input[idx], 'labels': self.labels[idx]}

# Instantiate the dataset and dataloader
num_samples = 1000
sequence_length = 10
feature_dim = 5
batch_size = 32

dataset = CustomDataset(num_samples, sequence_length, feature_dim)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example usage in a training loop
model = ConcatenatedModel(lstm_input_size=feature_dim, lstm_hidden_size=16, transformer_d_model=16, transformer_nhead=2, transformer_layers=2, fc_hidden_size=32, output_size=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for batch in dataloader:
    lstm_input, transformer_input, labels = batch['lstm_input'], batch['transformer_input'], batch['labels']

    # Assuming the target sequence for the transformer is the same as the input sequence
    transformer_tgt = transformer_input.clone()

    # Forward pass
    outputs = model(lstm_input, transformer_input, transformer_tgt)
    loss = criterion(outputs.squeeze(), labels.float())

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


TypeError: forward() takes 3 positional arguments but 4 were given

In [51]:
rnn = nn.LSTM(1, 4, 2)
input = torch.randn(5, 3, 1)
h0 = torch.zeros(2, 3, 4)
c0 = torch.zeros(2, 3, 4)
output, (hn, cn) = rnn(input)

In [52]:
hn.size()

torch.Size([2, 3, 4])

In [6]:
import torch
import torch.nn as nn

# Define input dimensions
batch_size = 212
sequence_length = 100
input_size = 2  # Assuming 2 features in each time step
hidden_size = 128  # Desired hidden state size
num_layers = 1  # Number of LSTM layers

# Create an LSTM layer
lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

# Create a random input tensor with the specified dimensions
batch_context = torch.randn(batch_size, sequence_length, input_size)

# Pass the input through the LSTM
output, (hn, cn) = lstm(batch_context)

# Print the output shape
print("Output Shape:", hn.shape)


Output Shape: torch.Size([1, 212, 128])


In [ ]:
Output Shape: torch.Size([212, 100, 128])